In [1]:
from __future__ import division
import glob
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
import time
from collections import defaultdict
import csv
from itertools import islice
import matplotlib.pyplot as plt
%matplotlib inline

# Define functions

In [2]:
def makeFreqDict(csv_file):
    #intialize dictionary
    aminoacids = 'ARNDCQEGHILKMFPSTWYV'
    freq_dict = {'Patient':[], 'Visit':[], 'ORF':[], 'Position':[], 'Coverage':[],
                 'A':[],'R':[],'N':[],'D':[],'C':[],'Q':[],'E':[],'G':[],'H':[],'I':[],
                 'L':[],'K':[],'M':[],'F':[],'P':[],'S':[],'T':[],'W':[],'Y':[],'V':[]}
    #csv to dataframe
    df = pd.read_csv(csv_file)
    for i, group in df.groupby('Position'):  
        #convert counts to percentages
        total = sum(group['Count'])
        group['Percent'] = group['Count']/total
        #print group
        #get values for each position
        patient = list(group['Patient'])[0]
        visit = list(group['Visit'])[0]
        orf = list(group['ORF'])[0]
        pos = list(group['Position'])[0]
        cov = total
        #fill up dictionary with values
        freq_dict['Patient'].append(patient)
        freq_dict['Visit'].append(visit)
        freq_dict['ORF'].append(orf)
        freq_dict['Position'].append(pos)
        freq_dict['Coverage'].append(cov)  
        #fill up dictionary with frequencies
        for aa in aminoacids:
            x = group[group['AA'] == aa]
            if x.shape[0] == 0:
                freq_dict[aa].append(0)
            else:
                freq_dict[aa].append(list(x['Percent'])[0])
    return freq_dict

def freqToDataframe(freq_dict):
    col_order = ['Patient','Visit','ORF','Position','Coverage',
             'A','R','N','D','C','Q','E','G','H','I',
             'L','K','M','F','P','S','T','W','Y','V']
    df = pd.DataFrame(freq_dict)[col_order]
    return df

# AA counts to frequency matrix for each sample

In [12]:
t = time.time()

csv_files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/AAcounts/*.csv')

for csv_file in csv_files:
    out_file = csv_file.replace('/AAcounts/','/PatientFrequencyMatrix/')
    out_file = out_file.replace('.AAcounts.','.PatientMatrix.')
    #print csv_file
    #print out_file
    d = makeFreqDict(csv_file)
    df = freqToDataframe(d)
    df.to_csv(out_file, index=False)
    
elapsed = time.time() - t
print elapsed

134.477634907


/Users/greg/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Get consensus sequences

In [31]:
mtx_files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/PatientFrequencyMatrix/*.csv')

cols = ['A','R','N','D','C','Q','E','G','H','I',
        'L','K','M','F','P','S','T','W','Y','V']

for mtx_file in mtx_files:
    mdf = pd.read_csv(mtx_file)
    if mdf.shape[0] == 72:
        a = mdf[cols].idxmax(axis=1, skipna=True)
        print ''.join(list(a))

MEPVDPSLEPWKHPGSQPRTPCTACYCKKVLLSLPSLLHKKRLRHLLWQEEAGTATKTSSRQRGSSSFSTKA
MDPVDPKLEPWKHPGSQPKTACTNCYLYKVLLSLPSLFHNKRLRHLLWQEKAKTETKNSSRRSNSSSFSIKA
MEPVDPSLEPWKHPGSQPKTACNSCYCKKVLLSLPSLFHNKRLRHLLWQEEAETATTSFSRQSDRSSFSIKA
MDPVDPKLEPWKHPGSQPRTACTKCYCKKVLLSLPSLFHNKSFRHLLWQEEAETATKNSSRQSDSSRFSTKA
MEPVDPRLEPWKHPGSQPETPCNACYCYKVLLSLSMVFHKKRLRHLLWQEEAGTATKTSSRQSGSSSSSTKA
MEPVDPSLEPWKHPGSQPKTACTNCYCKKVLLSLPSLFRNKRLRHLLWQEEAETATTSSSRQSDSSSFSTKA
MEPVDPRLEPWKHPGSQPKTACNNCYCKKVLLSLPSLLHKKRLRHLLWQEEAETATKTCSRQSGSSSLSIKA
MEPVDPSLEPWKHPGSQPKTPCTNCYCKTVLLSLPKVFCNKRLRHLLWQEEAETATNSSLRQSDSSRISIKA
MEPVDPSLEPWKHPGSQPKTACTLLLLQKVLLSLPSLLHNKRLRHLLWQEEAEQATKSSSRQSDSSSLSIKA
MEPVDPRLEPWKHPGSQPKTPCTNCYCKKMLLSLPSLFHKKRLRHLLWQEEAETATTISSRQRDSSSFSIKA
MDPVDPRLEPWKHPGSQPKTACNTCYCKTVLLSLPSLFHKKRLRHLLWQEEAETATKSFSRQRDSSRFSIKA
MEPVDPRLEPWKHPGSQPKTACTQCYCKKVLLSLPSLFHEKRLRHLLWQEEAGTATKISSRQEDSSSASIKA
MEPVDPRLEPWKHPGSQPKTACTACYCKKVLLSLSSLLCNKRLRHLLWQEEAETATKSGSRQSDSSSFSIRA
MDPVDPRLEPWKHPGSQPKTACTNCYCKKVLLSLPSLFHAKRLRHLLWQEE

# Matrix for each position

In [32]:
mtx_files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/PatientFrequencyMatrix/*.csv')
mtx_files[0]

'/Users/greg/Desktop/FullNeuroIllumina/PatientFrequencyMatrix/A0001-R09.PatientMatrix.csv'